<a href="https://colab.research.google.com/github/namoshi/colab/blob/master/pytorch_linalg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Creates inputs
np.random.seed(0)
mu_np = np.random.rand(4)
L = np.random.rand(4, 4)
# Covariance matrix sigma is positive-definite
sigma_np = L @ L.T + np.eye(4)
normal_noise_np = np.random.standard_normal(mu_np.size)

def multivariate_normal_sample_np(mu, sigma, normal_noise):
    return mu + np.linalg.cholesky(sigma) @ normal_noise

print("Random sample: ", 
      multivariate_normal_sample_np(mu_np, sigma_np, normal_noise_np))


Random sample:  [2.9502426  1.78518077 1.83168697 0.90798228]


In [3]:
import torch

def multivariate_normal_sample_torch(mu, sigma, normal_noise):
    return mu + torch.linalg.cholesky(sigma) @ normal_noise

# NumPy arrays are wrapped as tensors and share their memory
mu_torch = torch.from_numpy(mu_np)
sigma_torch = torch.from_numpy(sigma_np)
normal_noise_torch = torch.from_numpy(normal_noise_np)

multivariate_normal_sample_torch(mu_torch, sigma_torch, normal_noise_torch)

tensor([2.9502, 1.7852, 1.8317, 0.9080], dtype=torch.float64)

In [4]:
sqrt_sigma_det_np = np.sqrt(np.linalg.det(sigma_np))
sqrt_L_det_np = np.prod(np.diag(np.linalg.cholesky(sigma_np)))

print("|sigma|^0.5 = ", sqrt_sigma_det_np)

print("|L| = ", sqrt_L_det_np)

|sigma|^0.5 =  4.237127491242027
|L| =  4.237127491242028


In [5]:
sqrt_sigma_det_torch = torch.sqrt(torch.linalg.det(sigma_torch))
sqrt_L_det_torch = torch.prod(torch.diag(torch.linalg.cholesky(sigma_torch)))

print("|sigma|^0.5 = ", sqrt_sigma_det_torch)

print("|L| = ", sqrt_L_det_torch)

|sigma|^0.5 =  tensor(4.2371, dtype=torch.float64)
|L| =  tensor(4.2371, dtype=torch.float64)


In [6]:
import torch.utils.benchmark as benchmark

t0 = benchmark.Timer(
    stmt='torch.sqrt(torch.linalg.det(sigma))',
    globals={'sigma': sigma_torch})

t1 = benchmark.Timer(
    stmt='torch.prod(torch.diag(torch.linalg.cholesky(sigma)))',
    globals={'sigma': sigma_torch})

print(t0.timeit(100))

print(t1.timeit(100))

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
torch.sqrt(torch.linalg.det(sigma))
  32.24 us
  1 measurement, 100 runs , 1 thread
torch.prod(torch.diag(torch.linalg.cholesky(sigma)))
  12.65 us
  1 measurement, 100 runs , 1 thread




## Autograd







In [7]:
t = torch.tensor(((1, 2), (3, 4)), dtype=torch.float32, requires_grad=True)

inv = torch.linalg.inv(t)
inv.backward(torch.ones_like(inv))

print(t.grad)

tensor([[-0.5000,  0.5000],
        [ 0.5000, -0.5000]])


In [8]:
a = np.array(((1, 2), (3, 4)), dtype=np.float32)

inv_np = np.linalg.inv(a)

def inv_backward(result, grad):
    return -(result.transpose(-2, -1) @ (grad @ result.transpose(-2, -1)))
grad_np = inv_backward(inv_np, np.ones_like(inv_np))

print(grad_np)

[[-0.5  0.5]
 [ 0.5 -0.5]]
